# REST API z wykorzystaniem biblioteki Flask 

Aplikacja ta ma na celu realizację połączenia Client-Serwer

1. Utwórz nowy katalog `docker` i przejdź do katalogu.

```bash
mkdir docker
cd docker
```

3. Utwórz nowy plik `app.py`, który zawierać będzie kod naszej aplikacji www.
   
5. Zweryfikuj werjsę biblioteki Flask w swoim środowisku python - użyj pip list oraz narzędzia grep
   
6. Przykładowy kod aplikacji znajdziesz na stronie https://flask.palletsprojects.com/en/3.0.x/quickstart/
Zweryfikuj, czy kod zawiera polecenie uruchamiające serwer:
```python
if __name__ == '__main__':
    app.run()
```

Wprowadź przykładowy kod prostej aplikacji do pliku app.py

5. Uruchom polecenie

```bash
python app.py
```
Jeśli w kodzie nie ma polecenia uruchamiającego serwer, można wykrzystać polecenie (zgodnie z instrukcją)
```bash
python -m flask run
```

6. W nowym terminalu uruchom
```bash
curl localhost:5000
```

Biblioteka flask realizuje dodawanie podstron i komuniakcję z serwerem za pomocą funkcji przykrytych dekoratorem. Przykład decoratora:

1. definiujemy funkcję, której parametrem może być funkcja
2. definiujemy wewnętrzną funkcję (wszystko jest obiektem), która uruchamia funkcję z parametru i wyświetla info o jej działaniu

> Zadanie: Utwórz i zrealizuj kod dekoratora 
> Funkcje poda prowadzący ćwiczenia

In [ ]:
# TWOJ KOD 

In [ ]:
import subprocess
import requests

In [ ]:
%%file app1.py

from flask import Flask

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/hello')
def say_hello():
    return "Hello World"

@app.route('/')
def say_he():
    return "Hello z innej strony"

if __name__ == '__main__':
    app.run()

In [ ]:
# uruchom proces 
p = subprocess.Popen(["python", "app1.py"])

In [ ]:
adres_url = "http://127.0.0.1:5000/hello"
response = requests.get(adres_url)

# pobierz pole content z obiektu response

# TWOJ KOD TUTAJ 


In [ ]:
# podaj adres url który nie jest zdefiniowany w kodzie serwera 

adres_url = " "

response = requests.get(adres_url)

# TWOJ KOD POBIERAJĄCY ODPOWIEDŹ jeśli pole status_code ma wartość 200



Wykorzystaj metodę `kill()`  do obiektu `p` w celu zamknięcia procesu

In [ ]:
# TWOJ KOD



In [ ]:
# sprawdz czy po zabiciu procesu nadal się on uruchamia
response = ...


## Przekazywanie parametrów metodą GET z wykorzystaniem adresu url

In [ ]:
%%file app2.py

from flask import Flask
from flask import request

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/hello', methods=['GET'])
def say_hello():
    name = request.args.get("name", "") # tutaj leci str
    title = request.args.get("title", "")
    if name:
        resp = f"Hello {title} {name}" if title else f"Hello {name}"
    else:
        resp = f"Hello {title}" if title else "Hello"
    return resp

if __name__ == '__main__':
    app.run(port=5005)

In [ ]:
p = subprocess.Popen(["python", "app2.py"])

In [ ]:
response = requests.get("http://127.0.0.1:5005/hello")
response.content

dodaj zmienną do adresu wpisując `?name=....`

In [ ]:
response = requests.get("http://127.0.0.1:5005/hello?name=Sebastian")
response.content

Aby zamiast tekstu zwrócić json możesz wykorzystać 

```python
from flask import jsonify

def moja_f():
    ...
    return jsonify(reponse=resp)

Ciekawym i bardziej funkcyjnym rozwiązaniem dla modeli ML jest biblioteka [litServe](https://lightning.ai/litserve)


Zauważ pipeline, który jest zapisany w sposób funkcyjny. 

In [4]:
%%file app_lit.py
import litserve as ls

class SimpleLitAPI(ls.LitAPI):
    def setup(self, device):
        self.model1 = lambda x: x**2
        self.model2 = lambda x: x**3

    def decode_request(self, request):
        return request["input"]

    def predict(self, x):
        squared = self.model1(x)
        cubed = self.model2(x)
        output = squared + cubed
        return {"output": output}

    def encode_response(self, output):
        return {"output": output}

if __name__ == "__main__":
    api = SimpleLitAPI()
    server = ls.LitServer(api)
    server.run(port=5001)

Overwriting app_lit.py


In [3]:
!pip install uvicorn fastapi litserve -q

In [5]:
import subprocess

p = subprocess.Popen(["python", "app_lit.py"])

In [6]:
import requests

response = requests.post("http://127.0.0.1:5001/predict", json={"input": 4.0})
print(f"Status: {response.status_code}\nResponse:\n {response.text}")

Status: 200
Response:
 {"output":{"output":80.0}}


In [7]:
p.kill()

# konteneryzacja 

## Plik aplikacji app.py

In [ ]:
%%file app.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return "<h1>hello world</h1>"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

## plik requirements.txt w którym zamieścimy potrzebne biblioteki

In [ ]:
%%file requirements.txt
Flask==3.0.1

## Dockerfile 
- pobranie obrazu systemu z pythonem
- kopia pliku z wymaganymi bibliotekami
- instalacja wymaganych bibliotek w środowisku
- skopiowanie pliku aplikacji
- uruchomienie aplikacji

In [ ]:
%%file Dockerfile
FROM python:3.11-slim-buster

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY app.py .

ENV FLASK_APP=app

EXPOSE 5000
CMD ["flask", "run", "--host", "0.0.0.0", "--port", "5000"]

```bash
docker build -t test_hello .

docker run -p 5000:5000 test_hello
```